In [1]:
import pandas as pd
from src.training.bert_pipeline import TrainingBertPipeline
from src.training.bert_truncate_pipeline import BertTruncatePipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   question           4859 non-null   object 
 1   reference_answer   5363 non-null   object 
 2   answer             5363 non-null   object 
 3   score              5363 non-null   float64
 4   dataset            5363 non-null   object 
 5   normalized_score   5363 non-null   float64
 6   normalized_score2  5363 non-null   int64  
 7   bert_length        5363 non-null   int64  
 8   indobert_length    5363 non-null   int64  
 9   albert_length      5363 non-null   int64  
 10  longformer_length  5363 non-null   int64  
 11  multibert_length   5363 non-null   int64  
 12  indoalbert_length  5363 non-null   int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 544.8+ KB


In [3]:
df['dataset'].value_counts()
# sag [10]
# sci [10]
# cunlp [10][13]
# stita [10]
# analisis_essay [6]

dataset
sag               2558
analisis_essay    2162
stita              333
cunlp              171
sci                139
Name: count, dtype: int64

In [4]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results.csv"):
    df_result = pd.read_csv("experiments/results/results.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results.csv' does not exist.")

18


In [5]:
results = []
results_epoch = []

In [6]:
batch_sizes = [4]
epochs_list = [5, 10]
learning_rates = [1e-5, 5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [7]:
# model = [
#     ("bert_length", "bert-base-uncased"),
#     ("indobert_length", "indobenchmark/indobert-base-p1"),
#     ("albert_length", "albert-base-v1"),
#     ("indoalbert_length", "indobenchmark/indobert-lite-base-p2"),
#     ("longformer_length", "allenai/longformer-base-4096"),
#     ("multibert_length", "google-bert/bert-base-multilingual-cased")
# ]

In [8]:
for batch_size in batch_sizes:
    for num_epochs in epochs_list:
        for lr in learning_rates:
            df_result1 = None
            # Check if the second file exists
            if os.path.exists("experiments/results/results_epoch.csv"):
                df_result1 = pd.read_csv("experiments/results/results_epoch.csv")
                print(max(df_result1['valid_qwk']))
            else:
                print("File 'results_epoch.csv' does not exist.")
            config = {
                "df": df,
                "model_name": "bert-base-uncased",
                "batch_size": batch_size,
                "learning_rate": lr,
                "epochs": num_epochs,
                "config_id": idx,
                "max_seq_len": 512,
                "col_length": "bert_length",
                "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf")
            }

            logging.info(
                f"Running configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                f"max_seq_length={config['max_seq_len']}, epochs={num_epochs}, learning_rate={lr}"
            )
            
            print(
                f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                f"max_seq_length={config['max_seq_len']}, epochs={num_epochs}, learning_rate={lr}"
            )
            
            try:
                pipeline = BertTruncatePipeline(config, results, results_epoch)
                pipeline.run_training()

                # Save results
                results_path = os.path.join(ROOT_DIR, "experiments/results/results.csv")
                results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch.csv")
                TrainingBertPipeline.save_csv(results, results_path)
                TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
            except Exception as e:
                logging.error(f"Error in config_id={idx}: {str(e)}")
                print(f"Error in config_id={idx}: {str(e)}")
                torch.cuda.empty_cache()
            finally:
                # Clear GPU memory after every configuration
                del pipeline.model
                del pipeline.tokenizer
                del pipeline.optimizer
                torch.cuda.empty_cache()

            idx += 1

0.9029191176432184

Running configuration: config_id=19, model_name=bert-base-uncased, batch_size=4, max_seq_length=512, epochs=5, learning_rate=1e-05
split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/5 ======


c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0445, Train QWK: 0.6652, Train Pearson: 0.7752
Validation Loss: 0.0316, Validation QWK: 0.8515, Validation Pearson: 0.8632
====== Training Epoch 2/5 ======
Train Loss: 0.0239, Train QWK: 0.7533, Train Pearson: 0.8817
Validation Loss: 0.0236, Validation QWK: 0.8915, Validation Pearson: 0.8935
====== Training Epoch 3/5 ======
Train Loss: 0.0162, Train QWK: 0.7981, Train Pearson: 0.9216
Validation Loss: 0.0316, Validation QWK: 0.8712, Validation Pearson: 0.8866
====== Training Epoch 4/5 ======
Train Loss: 0.0134, Train QWK: 0.8146, Train Pearson: 0.9357
Validation Loss: 0.0327, Validation QWK: 0.8573, Validation Pearson: 0.8980
====== Training Epoch 5/5 ======
Train Loss: 0.0100, Train QWK: 0.8344, Train Pearson: 0.9521
Validation Loss: 0.0301, Validation QWK: 0.8666, Validation Pearson: 0.8975
Test Loss: 0.0331, Test QWK: 0.8496, Test Pearson: 0.8836
0.9029191176432184

Running configuration: config_id=20, model_name=bert-base-uncased, batch_size=4, max_seq_length=512, epoc

Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0436, Train QWK: 0.6706, Train Pearson: 0.7784
Validation Loss: 0.0368, Validation QWK: 0.8284, Validation Pearson: 0.8828
====== Training Epoch 2/5 ======
Train Loss: 0.0224, Train QWK: 0.7572, Train Pearson: 0.8900
Validation Loss: 0.0263, Validation QWK: 0.8818, Validation Pearson: 0.8896
====== Training Epoch 3/5 ======
Train Loss: 0.0144, Train QWK: 0.8056, Train Pearson: 0.9307
Validation Loss: 0.0268, Validation QWK: 0.8834, Validation Pearson: 0.8846
====== Training Epoch 4/5 ======
Train Loss: 0.0115, Train QWK: 0.8228, Train Pearson: 0.9450
Validation Loss: 0.0245, Validation QWK: 0.8856, Validation Pearson: 0.8926
====== Training Epoch 5/5 ======
Train Loss: 0.0086, Train QWK: 0.8390, Train Pearson: 0.9589
Validation Loss: 0.0278, Validation QWK: 0.8656, Validation Pearson: 0.9038
Test Loss: 0.0294, Test QWK: 0.8519, Test Pearson: 0.8884
0.9029191176432184

Running configuration: config_id=21, model_name=bert-base-uncased, batch_size=4, max_seq_length=512, epoc

Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0403, Train QWK: 0.6683, Train Pearson: 0.7961
Validation Loss: 0.0310, Validation QWK: 0.8673, Validation Pearson: 0.8743
====== Training Epoch 2/10 ======
Train Loss: 0.0229, Train QWK: 0.7641, Train Pearson: 0.8873
Validation Loss: 0.0260, Validation QWK: 0.8862, Validation Pearson: 0.8913
====== Training Epoch 3/10 ======
Train Loss: 0.0158, Train QWK: 0.7907, Train Pearson: 0.9235
Validation Loss: 0.0330, Validation QWK: 0.8738, Validation Pearson: 0.8917
====== Training Epoch 4/10 ======
Train Loss: 0.0126, Train QWK: 0.8220, Train Pearson: 0.9396
Validation Loss: 0.0328, Validation QWK: 0.8663, Validation Pearson: 0.9047
====== Training Epoch 5/10 ======
Train Loss: 0.0088, Train QWK: 0.8504, Train Pearson: 0.9580
Validation Loss: 0.0323, Validation QWK: 0.8539, Validation Pearson: 0.8922
====== Training Epoch 6/10 ======
Train Loss: 0.0073, Train QWK: 0.8673, Train Pearson: 0.9652
Validation Loss: 0.0237, Validation QWK: 0.8947, Validation Pearson: 0.8957
====== T

Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0444, Train QWK: 0.6717, Train Pearson: 0.7755
Validation Loss: 0.0357, Validation QWK: 0.8521, Validation Pearson: 0.8851
====== Training Epoch 2/10 ======
Train Loss: 0.0231, Train QWK: 0.7668, Train Pearson: 0.8868
Validation Loss: 0.0350, Validation QWK: 0.8644, Validation Pearson: 0.8690
====== Training Epoch 3/10 ======
Train Loss: 0.0171, Train QWK: 0.7956, Train Pearson: 0.9173
Validation Loss: 0.0237, Validation QWK: 0.8935, Validation Pearson: 0.8947
====== Training Epoch 4/10 ======
Train Loss: 0.0132, Train QWK: 0.8246, Train Pearson: 0.9368
Validation Loss: 0.0250, Validation QWK: 0.8840, Validation Pearson: 0.8925
====== Training Epoch 5/10 ======
Train Loss: 0.0085, Train QWK: 0.8381, Train Pearson: 0.9594
Validation Loss: 0.0283, Validation QWK: 0.8657, Validation Pearson: 0.9063
====== Training Epoch 6/10 ======
Train Loss: 0.0072, Train QWK: 0.8729, Train Pearson: 0.9658
Validation Loss: 0.0245, Validation QWK: 0.8878, Validation Pearson: 0.8896
====== T